In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

## Data

In [ ]:
# Iris dataset: https://en.wikipedia.org/wiki/Iris_flower_data_set
# Added some noise to make it harder to classify

FTS = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]
TARGET = "target"
ADD_NOISE = True

data, target = load_iris(return_X_y=True, as_frame=True)
data = pd.concat([data, target], axis=1)
if ADD_NOISE:
    np.random.seed(42)
    data[FTS] += np.random.normal(0, 1, data[FTS].shape)

data.head()

In [ ]:
data[TARGET].value_counts()

## Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data[FTS], data[TARGET], test_size=0.2, random_state=42
)

## Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=10000, random_state=42)
logreg.fit(X_train, y_train)

In [ ]:
def get_cm_scores(model, X_train, y_train, X_test, y_test):
    _, axs = plt.subplots(1, 2, figsize=(20, 5))
    for i, (name, X, y) in enumerate(
        [("train", X_train, y_train), ("test", X_test, y_test)]
    ):
        report = classification_report(y, model.predict(X), output_dict=True)[
            "macro avg"
        ]
        report["accuracy"] = model.score(X, y)
        report = {k: round(v, 3) for k, v in report.items()}
        sns.heatmap(
            confusion_matrix(y, model.predict(X)), annot=True, fmt="d", ax=axs[i]
        ).set_title(name + str(report))

In [ ]:
get_cm_scores(logreg, X_train, y_train, X_test, y_test)

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier(random_state=42, max_depth=100)
tree.fit(X_train, y_train)

In [ ]:
get_cm_scores(tree, X_train, y_train, X_test, y_test)

## Explainability

In [ ]:
logreg.coef_, logreg.intercept_  # coefficients and intercepts

In [ ]:
importances = pd.DataFrame(
    {"feature": FTS, "importance": np.abs(logreg.coef_).mean(axis=0)}
)
plt.barh(importances.feature, importances.importance)
plt.title("Feature importances Logistic Regression")

In [ ]:
importances = pd.DataFrame({"feature": FTS, "importance": tree.feature_importances_})
plt.barh(importances.feature, importances.importance)
plt.title("Feature importances Decision Tree")

In [ ]:
_, ax = plt.subplots(figsize=(40, 20))
plot_tree(
    tree,
    feature_names=FTS,
    class_names=data[TARGET].unique().astype(str),
    filled=True,
    rounded=True,
    fontsize=14,
    ax=ax,
)
plt.show()